Hi Zach, from looking through GEODYN, it looks like the drag acceleration is computed in the True-of-Date frame rather than (as I had assumed) in the Satellite Body Fixed (SBF). So it will be a little harder to print out the velocity components in the SBF frame. Not impossible though... It looks like panel 1 has a normal vector in the SBF X direction, panel 2's normal vector is in the SBF Y, and panel 4's is in the SBF Z direction. So you could use a temporary print statement to print CTHETA (MODDRIA.F90: line 234) for each of those three panels. This would give you the unit vector of the velocity in the SBF frame ( V_unit_SBF = [CTHETA(panel 1) ; CTHETA(panel 2) ; CTHETA(panel 4)] ).
...you should double check that panels 1, 2, and 4 are in the respective +X, +Y, and +Z directions though; if not, identify 3 panels that are.

Zach, for the moveable panels, we won't be able to get the full SBF x/y/z components of velocity (since there aren't panels that face directly in all three of the X/Y/Z directions), but printing out CTHETA for panel 14 (that has a normal vector in the +Z direction) will probably give us enough information to go on.

In [1]:
import pandas as pd

arc_timesfile = '/data/SatDragModelValidation/data/inputs/raw_inputdata/data_ICESat2/arc_times.txt'


### Make a function that finds the right range of dates in this file.
arcs = pd.read_csv(arc_timesfile, 
            sep = ',',
#             dtype=object,
            names = [
                'arc'         ,
                'epoch_start' ,
                'epoch_stop'  ,
                'orbit_start' ,
                'orbit_stop'  ,
                    ],)


arcs_yyyyddd = [x.strip() for x in arcs['arc'].values.tolist()]
epochstart   = [x.strip() for x in arcs['orbit_start'].values.tolist()]
epochstop    = [x.strip() for x in arcs['orbit_stop' ].values.tolist()]


In [2]:
from pygeodyn.PYGEODYN import Pygeodyn
import datetime


run_list = [
                'msis2',
#                 'tiegcm_oc',
#                 'ctipe_oc',
#                 'jb2008',
#                 'dtm2020_o',
#                 'gitm',
#                 'hasdm_oc',
           ]

dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
run_dict={}
for i in run_list:
    if i =='msis2':
        run_dict[i]={}
        run_dict[i]['num'] = 5
        run_dict[i]['model_path'] = None
    if i =='dtm2020_o':
        run_dict[i]={}
        run_dict[i]['num'] = 3
        run_dict[i]['model_path'] = None
    if i =='jb2008':
        run_dict[i]={}
        run_dict[i]['num'] = 1
        run_dict[i]['model_path'] = None
        
print(run_dict)

{'msis2': {'num': 5, 'model_path': None}}


In [3]:
# 299-303


In [4]:
%load_ext autoreload
%autoreload 2
from gc import collect as gc_collect
import pickle 
import os

# file_raw_ICs = f"/data/SatDragModelValidation/data/inputs/sat_icesat2/g2b/"\
#           +"ICESat2_RawEphem_2018_10.txt"

g2b_path = "/data/SatDragModelValidation/data/inputs/sat_icesat2/g2b/"
dir_save    =  '/data/SatDragModelValidation/data/outputs_clean/'\
             + 'icesat2/O2R2023_longimeperiod/1_DRIAruns/'

# month_list = [, 'nov', 'dec']
# month_list = ['oct', 'nov']
month_list = ['oct']

obj = {}

for imonth,month in enumerate(month_list):
    if month=='oct':
        m_num = 10
    if month=='nov':
        m_num = 11
    if month=='dec':
        m_num = 12

    file_raw_ICs = f"{g2b_path}ICESat2_RawEphem_2018_{m_num}.txt"
    file_g2b     = f"pce_icesat2_pso_2018_{m_num}"
    
    input_arcs       = []
    input_epochstart = []
    input_epochstop  = []
    
    for i,val in enumerate(arcs_yyyyddd):
        if pd.to_datetime(val[:8],format="%Y.%j").month == m_num:
    #         print(val, i)
            input_arcs.append(      arcs_yyyyddd[i])
            input_epochstart.append(epochstart[i])
            input_epochstop.append( epochstop[i])
    
    A_arcs   = input_arcs[12:19]
    A_starts = input_epochstart[12:19]
    A_stops  = input_epochstop[12:19]
    
    for i,val in enumerate(A_arcs):
        print(f"{i:3} {A_arcs[i]:10}| {pd.to_datetime(A_arcs[i][:8],format='%Y.%j').strftime('%b %d')}  , {A_starts[i]:20},  {A_stops[i]:20}")
#     print(f"{i} {arcs_yyyyddd[i]:10}  , {epochstart[i]:20},  {epochstop[i]:20}")

    for i,den in enumerate(run_list):

        settings_icesat2= {# Basic input settings
                     'satellite'      : {'input': 'icesat2'},
                     'den_model'      : {'input': den},
                     'run_type'       : {'input': 'DataReduction_PCE'},
                     'run_specifier'  : {'input': '_'+month},
                     'cd_model'       : {'input': 'DRIA'},
    #                  'cd_model'       : {'input': 'BWDRAG'},
                     'file_string'    : {'input': 'test'},
                     'model_data_path' : {'input': run_dict[den]['model_path']},
                     # Force Model settings
                      'cd_type'               : {'input':"Fixed_CD"},
                      'cd_value'              : {'input':2.500000},
                      'scaling_factor'        : {'input':False},
    #                   'hours_between_cd_adj'  : {'input':24},
                      #### Comment for unadjusted run:
                      'cd_adjustment_boolean' : {'input':False },
            #### DRIA CD Model Parameters
            'cd_model_params' : {'input':{ 
                        'MS'     : '26.980D0'   ,  #!  molar mass for each panel (g/mol)
                        'TW'     : '300.0D0'    ,  #!  temperature of panels  (K)
                           ###  Alpha should be some value from 0 to 1
                        'ALPHA'  : '0.890D0'    ,  #!  accomodation coefficient
                        'KL'     : '0.0D0'    ,  #!  langmuir parameter
                        'FRACOX' : '1.0D0'   ,  #!  fraction of surface covered by atomic oxygen
                       }},

                      #### ---------------------------------------
                     # Run
                      'arc_type'       : {'input':'Nominal30hr_and_AB'},      
                      'step'           : {'input': 60.},
                      'orbfil_step'    : {'input': 120.},
                      'which_ICfile'   : {'input':file_raw_ICs},
                      'which_g2bfile'  : {'input':file_g2b},
                        #
                      'arc'            : {'input':A_arcs    },
                      'epoch_start'    : {'input': A_starts },
                      'epoch_stop'     : {'input':A_stops   },  
                       #                                
                      'global_options' : {'input':'pso_2018'},
                     # Request read on raw outputs
                      'request_data'   : {'input': ['Trajectory_orbfil', 
                                                   'Density', 
                                                   'Residuals_summary',
                                                   'DragFile',
                                                   'AdjustedParams'
                                                   ]},
                  #end dict
                  }




        sat = Pygeodyn(settings_icesat2, use_file=False)
        sat.run_arcs()
#         obj[den] =  sat.getData_BigData_lowmemory()
#         obj[den] = vars(obj[den])
        gc_collect()


#     pickleName = f'_{month}_DRIAtest.pkl'

#     pickle_file = dir_save+den+pickleName
#     if not os.path.exists(pickle_file):
#         print('Must create pickle file...')
#         print('   ',  pickle_file)
#         print('   ', 'Reading Geodyn Data')

#         ### Load the data into an object
#         sat = Pygeodyn(settings_icesat2, use_file=False)
#         obj = sat.getData_BigData_lowmemory()
#         gc_collect()

#         #### Pickle the object to save it
#         print('   ', 'Saving pickle')
#         filehandler = open(pickle_file, 'wb') 
#         pickle.dump(vars(obj), filehandler)
#         filehandler.close()
#         obj = 0
#         print('   ', 'Saved pickle')

# obj = {}
# for i,model in enumerate(run_list):
#     for imonth,month in enumerate(month_list):
#         pickleName = f'_{month}_DRIAtest.pkl'

#         ### Load the data if the pickles exist
#         print()
#         print()
#         gc_collect()

#         pickle_file = dir_save+model+pickleName

#         filehandler = open(pickle_file, 'rb') 
#         obj[month+model] = pickle.load(filehandler)
#         filehandler.close()
#         print('Loaded data from pickle... ',  model)
    
    
# ## -----------------------------------------------------
# ## startdate = "2018-10-14"
# ## enddate   = "2018-11-02"  
# ## sat = Pygeodyn(settings_icesat2, use_file=False)
# ## sat.initialize_timeperiod_stage1(startdate, enddate,
# ##                                  overwrite_exat=False, 
# ##                                  overwrite_ICtext=False)
# ## import sys
# ## sys.exit(0)

  0 2018.299  | Oct 26  , 2018-10-25 21:18:00 ,  2018-10-27 02:42:00 
  1 2018.300  | Oct 27  , 2018-10-26 21:18:00 ,  2018-10-28 02:42:00 
  2 2018.301A | Oct 28  , 2018-10-27 21:18:00 ,  2018-10-28 16:00:00 
  3 2018.301B | Oct 28  , 2018-10-28 17:36:00 ,  2018-10-29 02:42:00 
  4 2018.302A | Oct 29  , 2018-10-28 21:18:00 ,  2018-10-29 10:18:00 
  5 2018.302B | Oct 29  , 2018-10-29 11:42:00 ,  2018-10-30 02:42:00 
  6 2018.303  | Oct 30  , 2018-10-29 21:18:00 ,  2018-10-31 02:42:00 
Using the ICESat-2 Class
Run #1     Current Time =      12:01:53  GMT-7
Run #1
|————————————————————————————————————————————————————————————————————————————————
| Run #1 Parameters
| —————————————————
|  Run Specs
|  --------- 
|    Satellite    icesat2
|    Run Type     DataReduction_PCE
|    CD Type      Fixed_CD
|    CD Value     2.5
|    Density      msis2
|
|  Epoch Info 
|  ---------- 
|    Arc          2018.299
|    Arc length   29.4 hours
|    Epoch Start  2018-10-25 21:18:00
|    Epoch End    201


Run #6          Running IIS
Run #6          No errors in IIS
Run #6 ---------End of IIS

Run #6          Running IIE
Run #6          Current Time = 12:10:12 GMT-7
 ------ Current DIR:  /data/SatDragModelValidation/data/tmp/msis2_DRIA_oct/icesat2_2018302B_msis2.test
Run #6          No errors in IIE
Run #6 ---------End of IIE
Run #6          Time of IIE:  41.52057385444641 secs ( 0.6920095642407735  mins)
Run #6          Current Time = 19:10:54
Saving fort.103 as drag_file
Saving fort.104 as SatGeometry_file
Run #6                Finished renaming files
Run #6                Finished copying files to outputdir
Run #7     Current Time =      12:10:57  GMT-7
Run #7
|————————————————————————————————————————————————————————————————————————————————
| Run #7 Parameters
| —————————————————
|  Run Specs
|  --------- 
|    Satellite    icesat2
|    Run Type     DataReduction_PCE
|    CD Type      Fixed_CD
|    CD Value     2.5
|    Density      msis2
|
|  Epoch Info 
|  ---------- 
|    Arc     

In [5]:
# pd.to_datetime(input_arcs[6],format="%Y.%j").strftime('%b %d')

In [6]:
self_year = 2018
import time
from datetime import datetime,timedelta
import numpy as np

def read_drag_file(_drag_filename):
    '''
         Read the drag file.  
         The drag file has a different date format than the other outputs
                   so it is dealt with here in the method.

    '''
    start = time.time()

    ##### Unzip the density file:


    #### The density file is a very simple format
    #### with this it can be loaded using pd.csv very easily
    Drag_csv = pd.read_csv(_drag_filename, 
    #                             skiprows = 1, 
                            names = ['YYMMDD',
                                     'HHMMSS',
                                    #                                      
                                     'CD',
                                     'TOTAREA',
                                     'VELREL',
                                     'SpeedRatio',
                                    #                                      
    #                                 'DXDD_X',
    #                                 'DXDD_Y',
    #                                 'DXDD_Z',
                                     "ctheta_1",
                                     "ctheta_2",
                                     "ctheta_4",
                                     "ctheta_14"

                                  ],
                            sep = '\s+',
                            )
    drag_df = pd.DataFrame(Drag_csv)

    fix_D_decimal_to_E = {}

    for colname in list(drag_df.columns)[2:]:
#         print(colname)
        fix_D_decimal_to_E[colname] = []

        for i,val in enumerate(drag_df[colname]):
            list_val1 = list(val)
            indx1 = list(val).index('D')
            list_val1[indx1] = 'E'
            list_val1 = "".join(list_val1)

            #### If you get an error here, it is likely:
            ####  due to some values not being floats?
            ###
            val_float1 = float(list_val1)

            fix_D_decimal_to_E[colname].append(val_float1)

        drag_df[colname] = fix_D_decimal_to_E[colname]


    ####--------------------------------------------------------
    #### Now we must correct the formatting of the HoursMinutesSeconds column
    #### The lack of zeros was really messing things up, so I forced them in there...
    #### It works so I hope everyone is okay with how dumb it is.
    timeHHMMSS = [] 
    for i,val in enumerate(drag_df['HHMMSS'].values.astype(int)):
        # print(len(str(val)))
        if len(str(val)) == 1:
            timehhmmss_val = '00000'+ str(val)
            timeHHMMSS.append(timehhmmss_val)
        elif len(str(val)) == 2:
            timehhmmss_val = '0000'+ str(val)
            timeHHMMSS.append(timehhmmss_val)
        elif len(str(val)) == 3:
            timehhmmss_val = '000'+ str(val)
            timeHHMMSS.append(timehhmmss_val)
        elif len(str(val)) == 4:
            timehhmmss_val = '00'+ str(val)
            timeHHMMSS.append(timehhmmss_val)
        elif len(str(val)) == 5:
            timehhmmss_val = '0'+ str(val)
            timeHHMMSS.append(timehhmmss_val)
        else:
            timeHHMMSS.append(str(val))
    drag_df['timeHHMMSS'] = timeHHMMSS
    #--------------------------------------------------------
    #### Here we split up the years, months, and days.
    #### we add a '20' in front of the year
    # TODO:  need to find a workaround on this dumb YR variable being passed in.

    YR = int(str(self_year)[-2:])

    YYMMDD_list = drag_df['YYMMDD'].astype(int).astype(str)
    timeHHMMSS_list = drag_df['timeHHMMSS'].astype(str)

    if YR < 10:
        year    = ['200' + x[:1]  for x in YYMMDD_list]
        month   = [        x[1:3] for x in YYMMDD_list]
        day     = [        x[3:]  for x in YYMMDD_list]
        hours   = [        x[:2]  for x in timeHHMMSS_list]
        minutes = [        x[2:4] for x in timeHHMMSS_list]
        secs    = [        x[4:]  for x in timeHHMMSS_list]
    else:
        year    = ['20' + x[:2]  for x in YYMMDD_list]
        month   = [       x[2:4] for x in YYMMDD_list]
        day     = [       x[4:]  for x in YYMMDD_list]
        hours   = [       x[:2]  for x in timeHHMMSS_list]
        minutes = [       x[2:4] for x in timeHHMMSS_list]
        secs    = [       x[4:]  for x in timeHHMMSS_list]
    #--------------------------------------------------------
    drag_df['year']  = year
    drag_df['month'] = month
    drag_df['day']   = day
    drag_df['hours']  = hours
    drag_df['minutes'] = minutes
    drag_df['secs']  = secs
    #--------------------------------------------------------
    year= list(map(int, drag_df['year'].values))
    month= list(map(int, drag_df['month'].values))
    day= list(map(int, drag_df['day'].values))
    hour= list(map(int, drag_df['hours'].values))
    minute = list(map(int, drag_df['minutes'].values))
    second = list(map(int, drag_df['secs'].values))

    DATE = list(map(datetime, year,month, day, hour,minute,second ))

    #self.DEN_df['Date']  = DATE
    drag_df.insert(0, 'Date', DATE)

    #### DELETE the superfluous columns in the dataframe now.       
    del drag_df['year']
    del drag_df['month']
    del drag_df['day']
    del drag_df['hours']
    del drag_df['minutes']
    del drag_df['secs']
    del drag_df['timeHHMMSS']
    del drag_df['YYMMDD']
    del drag_df['HHMMSS']

    ### DELETE the duplicated dataset
    #### Find the index for the correct date and 
#         vals  = np.arange(drag_df.index[0],drag_df.index[-1]+1)
#         df = drag_df.set_index('Date',drop=False ) 
#         df['i_vals'] = vals
#         index_date = df.loc[df.index.max()]['i_vals'].min() 
#         for name in drag_df.columns:
#             drag_df[name] = drag_df[name][:index_date]

#         #### Drop the NaNs
#         drag_df = drag_df.dropna()
    # def nearest(items, pivot):
    #     return min(items, key=lambda x: abs(x - pivot))
    # epoch_start = self.prms_arc['start_ymdhms']
    # epoch_start_YYMMDD     = epoch_start[:6].strip() 
    # epoch_start_HHMM       = epoch_start[7:11].strip()
    # epoch_start_dt = pd.to_datetime( epoch_start_YYMMDD+epoch_start_HHMM, format='%y%m%d%H%M%S')
    # vals  = np.arange(drag_df.index[0],drag_df.index[-1]+1)
    # df = drag_df.set_index('Date',drop=False ) 
    # df['i_vals'] = vals
    # date_nearstart = nearest(pd.to_datetime(df['i_vals'].index), epoch_start_dt)

    # index_date = df['i_vals'][date_nearstart].min() #df['i_vals'][date_nearend].min()#df.loc[df.index.max()]['i_vals'].min() 
    # for name in drag_df.columns:
    #     drag_df[name] = drag_df[name][index_date:]

    # drag_df = drag_df.dropna()
    #--------------------------------------------------------

    drag_df = drag_df.drop_duplicates(subset=["Date"], keep='first'\
            ).sort_values(by='Date'\
                            ).reset_index(drop=True)


    drag_df = drag_df.dropna()



    end = time.time()
    elapsed = end - start
#         print('Density file end: ',elapsed)
    return(drag_df)



In [7]:
A_arcs

['2018.299',
 '2018.300',
 '2018.301A',
 '2018.301B',
 '2018.302A',
 '2018.302B',
 '2018.303']

In [8]:
path     = "/data/SatDragModelValidation/data/outputs_raw/icesat2/msis2/msis2_DRIA_oct/DENSITY"


for i,val in enumerate(A_arcs):
    day = "".join(val.split("."))
    filename = f"icesat2_{day}_msis2.testdrag_file"

    _drag_filename = f"{path}/{filename}"
    print(_drag_filename)

    if i == 0:
        drag_df = read_drag_file(_drag_filename)
    else:
        drag_df_2 = read_drag_file(_drag_filename)
        drag_df = drag_df.append(drag_df_2, ignore_index=True)
#         df1.append(df2)

/data/SatDragModelValidation/data/outputs_raw/icesat2/msis2/msis2_DRIA_oct/DENSITY/icesat2_2018299_msis2.testdrag_file
/data/SatDragModelValidation/data/outputs_raw/icesat2/msis2/msis2_DRIA_oct/DENSITY/icesat2_2018300_msis2.testdrag_file
/data/SatDragModelValidation/data/outputs_raw/icesat2/msis2/msis2_DRIA_oct/DENSITY/icesat2_2018301A_msis2.testdrag_file
/data/SatDragModelValidation/data/outputs_raw/icesat2/msis2/msis2_DRIA_oct/DENSITY/icesat2_2018301B_msis2.testdrag_file
/data/SatDragModelValidation/data/outputs_raw/icesat2/msis2/msis2_DRIA_oct/DENSITY/icesat2_2018302A_msis2.testdrag_file
/data/SatDragModelValidation/data/outputs_raw/icesat2/msis2/msis2_DRIA_oct/DENSITY/icesat2_2018302B_msis2.testdrag_file
/data/SatDragModelValidation/data/outputs_raw/icesat2/msis2/msis2_DRIA_oct/DENSITY/icesat2_2018303_msis2.testdrag_file


In [9]:
# drag_df_i
drag_df

,Date,CD,TOTAREA,VELREL,SpeedRatio,ctheta_1,ctheta_2,ctheta_4,ctheta_14
0,2018-10-25 21:14:09,3.362969,3.968812,7658.722829,6.864514,0.999325,0.036400,-0.005044,-0.016046
1,2018-10-25 21:15:09,3.345919,3.969474,7658.611534,6.953807,0.999323,0.036401,-0.005320,-0.016045
2,2018-10-25 21:16:09,3.331878,3.970238,7658.322138,7.028409,0.999321,0.036401,-0.005630,-0.016045
3,2018-10-25 21:17:09,3.320980,3.971078,7657.848625,7.086543,0.999320,0.036395,-0.005942,-0.016050
4,2018-10-25 21:18:09,3.313328,3.971736,7657.189727,7.127817,0.999318,0.036401,-0.006239,-0.016045
...,...,...,...,...,...,...,...,...,...
8691,2018-10-31 02:41:09,2.536419,22.859375,7644.801295,6.138763,0.999329,0.036410,-0.004050,0.999337
8692,2018-10-31 02:42:09,2.536558,22.858962,7645.317978,6.139286,0.999329,0.036431,-0.003829,0.999336
8693,2018-10-31 02:43:09,2.536811,22.858450,7645.974188,6.136736,0.999330,0.036416,-0.003655,0.999337
8694,2018-10-31 02:44:09,2.537200,22.858001,7646.749866,6.130138,0.999331,0.036411,-0.003483,0.999337


In [10]:
# drag_df_2

In [11]:
from pygeodyn.pygeodyn_plot_scalingfactors import *

coldict = {}
coldict['msis2']     = "#2ca02c"  # 'tab:green'
coldict['dtm2020_o'] = "#d62728"  # 'tab:red'
coldict['jb2008']    = "orange"   
coldict['hasdm_oc']  = "#1f77b4"     

tab_blue = "#1f77b4"


In [23]:
indplot = 2
model_m1 = den


fig  = make_subplots(
    rows=7, cols=1,
#         subplot_titles=([
#                      'Panel #1  [1,0,0]' , 
#                      'Panel #2  [0,1,0]' ,   
#                      'Panel #4  [0,0,1]' , 
#                      'Panel #14 [0,0,1]' ,
#                     ]),
    vertical_spacing = 0.035,
    shared_xaxes=True)


# 
fig.add_trace(go.Scattergl(x=drag_df['Date'][::indplot],
                           y=drag_df['ctheta_1'][::indplot],
                         mode='markers',
                         opacity=0.5,
                         marker=dict(color= tab_blue,size=3,),
                         showlegend=False,),
                         secondary_y=False,
                         row=1, col=1)
fig.add_trace(go.Scattergl(x=drag_df['Date'][::indplot],
                           y=drag_df['ctheta_2'][::indplot],
                         mode='markers',
                         opacity=0.5,
                         marker=dict(color= tab_blue,size=3,),
                         showlegend=False,),
                         secondary_y=False,
                         row=2, col=1)
fig.add_trace(go.Scattergl(x=drag_df['Date'][::indplot],
                           y=drag_df['ctheta_4'][::indplot],
                         mode='markers',
                         opacity=0.5,
                         marker=dict(color= tab_blue,size=3,),
                         showlegend=False,),
                         secondary_y=False,
                         row=3, col=1)
fig.add_trace(go.Scattergl(x=drag_df['Date'][::indplot],
                           y=drag_df['ctheta_14'][::indplot],
                         mode='markers',
                         opacity=0.5,
                         marker=dict(color= tab_blue,size=3,),
                         showlegend=False,),
                         secondary_y=False,
                         row=4, col=1)
fig.add_trace(go.Scattergl(x=drag_df['Date'][::indplot],
                           y=drag_df['CD'][::indplot],
                         mode='markers',
                         opacity=0.5,
                         marker=dict(color= 'black',size=3,),
                         showlegend=False,),
                         secondary_y=False,
                         row=5, col=1)

fig.add_trace(go.Scattergl(x=drag_df['Date'][::indplot],
                           y=drag_df['TOTAREA'][::indplot],
                         mode='markers',
                         opacity=0.5,
                         marker=dict(color= 'black',size=3,),
                         showlegend=False,),
                         secondary_y=False,
                         row=6, col=1)


fig.add_trace(go.Scattergl(x=drag_df['Date'][::indplot],
                           y=drag_df['CD'][::indplot]*drag_df['TOTAREA'][::indplot],
                         mode='markers+lines',
                         opacity=0.5,
                         marker=dict(color= 'black',size=3,),
                           line = dict(color='black', dash ='solid',  width=2),
                         showlegend=False,),
                         secondary_y=False,
                         row=7, col=1)
        
    
fig.update_yaxes(title_text="CTHETA 1", 
                 exponentformat= 'power',row=1, col=1)
fig.update_yaxes(title_text="CTHETA 2", 
                 exponentformat= 'power',row=2, col=1)
fig.update_yaxes(title_text="CTHETA 4", 
                 exponentformat= 'power',row=3, col=1)
fig.update_yaxes(title_text="CTHETA 14", 
                 exponentformat= 'power',row=4, col=1)
fig.update_yaxes(title_text="CD", 
                 exponentformat= 'power',row=5, col=1)
fig.update_yaxes(title_text="TOTAREA", 
                 exponentformat= 'power',row=6, col=1)
fig.update_yaxes(title_text="CD*TOTAREA", 
                 exponentformat= 'power',row=7, col=1)

#     fig.update_yaxes(title_text="Rel. Velocity [m/s] ", 
#                      exponentformat= 'power',row=5, col=1)
#     fig.update_yaxes(title_text="Speed Ratio", 
#                      exponentformat= 'power',row=6, col=1)




#######################################################
font_dict=dict(family='Arial',size=11,color='black')
## automate the specification of the axes for subplots
rownum, colnum = fig._get_subplot_rows_columns()
for i in rownum:
    if len(rownum)==1:
        L_ticklabel = True
    else:
        if i < len(rownum):
            L_ticklabel = True
        else:
            L_ticklabel = True
    fig.update_xaxes(### LINE at axis border
                      showline=True,
                      showticklabels=L_ticklabel,
                      tickformat= '%m/%d',
                      linecolor='black',
                      linewidth=1,
                     ### Major ticks
                      ticks='inside',
                      tickfont=font_dict,
                      mirror=True,
#                       tickwidth=2,
#                       ticklen=9,
                      tickcolor='grey',
#                       tick0="2018-11-9" ,
#                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
                      #### Minor Ticks
                       minor=dict(
                         dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,
                         ticklen=4,
                         tickcolor='grey',
                         ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',
                       gridwidth=1,
                       layer='above traces',
                       tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                         showticklabels=True,
                         linecolor='black',  # line color
                         linewidth=1,        # line size
                     ticks='inside',     # ticks outside axis
                     tickfont=font_dict, # tick label font
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=1,      # tick width
                     tickcolor='black',  # tick color
                     gridcolor='gainsboro',
                     gridwidth=1,
                     layer='above traces',
                     row=i, col=1)

    
fig.update_layout(
#                   title = '',
                  autosize=False,    width=1000,    height=1100,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict,
                  plot_bgcolor='white', 
                 )




#### Missing data days
fig.add_vrect(x0="2018-10-28 16:00:00" ,
              x1="2018-10-28 17:36:00" ,
                fillcolor="red",opacity=0.5,line_width=0)

fig.add_vrect(x0="2018-10-29 10:18:00" ,
              x1="2018-10-29 11:42:00" ,
                fillcolor="red",opacity=0.5,line_width=0)


fig.add_vrect(x0="2018-10-25 21:18:00" ,
              x1="2018-10-27 13:00:00" ,
                fillcolor="pink",opacity=0.25,line_width=0,
                annotation_text = 'SAILBOAT',annotation_position="bottom left",)
fig.add_vrect(x0="2018-10-27 13:00:00" ,
              x1="2018-10-31 02:42:00" ,
                fillcolor="green",opacity=0.25,line_width=0,
                annotation_text = 'AIRPLANE',annotation_position="bottom right",)

fig.show(config=config)



In [ ]:
#   0 2018.299  | Oct 26  , 2018-10-25 21:18:00 ,  2018-10-27 02:42:00 
#   1 2018.300  | Oct 27  , 2018-10-26 21:18:00 ,  2018-10-28 02:42:00 
#   2 2018.301A | Oct 28  , 2018-10-27 21:18:00 ,  2018-10-28 16:00:00 
#   3 2018.301B | Oct 28  , 2018-10-28 17:36:00 ,  2018-10-29 02:42:00 
#   4 2018.302A | Oct 29  , 2018-10-28 21:18:00 ,  2018-10-29 10:18:00 
#   5 2018.302B | Oct 29  , 2018-10-29 11:42:00 ,  2018-10-30 02:42:00 
#   6 2018.303  | Oct 30  , 2018-10-29 21:18:00 ,  2018-10-31 02:42:00 

#noon on the 27th is the transition from sailboat to airplane

In [13]:
# indplot = 2
# model_m1 = den


# fig  = make_subplots(
#     rows=2, cols=1,
#         subplot_titles=([
#                      'CD' , 
#                      'CD*TOTAREA' ,   
# #                      'Panel #4  [0,0,1]' , 
# #                      'Panel #14 [0,0,1]' ,
#                     ]),
#     vertical_spacing = 0.05,
#     shared_xaxes=True)


# # 
# fig.add_trace(go.Scattergl(x=drag_df['Date'][::indplot],
#                            y=drag_df['CD'][::indplot],
#                          mode='markers',
#                          opacity=0.5,
#                          marker=dict(color= coldict[model_m1],size=3,),
#                          showlegend=False,),
#                          secondary_y=False,
#                          row=1, col=1)
# fig.add_trace(go.Scattergl(x=drag_df['Date'][::indplot],
#                            y=drag_df['CD'][::indplot]*drag_df['TOTAREA'][::indplot],
#                          mode='markers',
#                          opacity=0.5,
#                          marker=dict(color= coldict[model_m1],size=3,),
#                          showlegend=False,),
#                          secondary_y=False,
#                          row=2, col=1)
        
    
# fig.update_yaxes(title_text="CD", 
#                  exponentformat= 'power',row=1, col=1)
# fig.update_yaxes(title_text="Total Area", 
#                  exponentformat= 'power',row=2, col=1)

# #     fig.update_yaxes(title_text="Rel. Velocity [m/s] ", 
# #                      exponentformat= 'power',row=5, col=1)
# #     fig.update_yaxes(title_text="Speed Ratio", 
# #                      exponentformat= 'power',row=6, col=1)




# fig.update_layout(
# #                   title = f"Evolution of CTHETA[i]=dot(TDNRM[x,y,z][i], XVEL[x,y,z][i]), sailboat mode ",
#                   autosize=False,    width=1000,    height=500,
# #                   legend= {'itemsizing': 'trace'},
# #                   font=font_dict,
# #                   plot_bgcolor='white', 
#                  )
# fig.update_annotations(font_size=16)  # Increase size of subplot title





# fig.show(config=config)



